In [20]:
import os
import librosa
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

train_audio_path = '../data/'
labels=["stop", "three", "two", "wow", "zero"]

all_wave = []
all_label = []
for label in labels:
    print(label)
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples)== 8000) : 
            all_wave.append(samples)
            all_label.append(label)

stop
three
two
wow
zero


In [24]:
import numpy as np

le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)

y=np_utils.to_categorical(y, num_classes=len(labels))
all_wave = np.array(all_wave).reshape(-1,8000,1)

In [46]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

base_model = load_model('speech_assistant_model.hdf5')
for layer in base_model.layers[:-4]:
    layer.trainable = False;
base_model.summary()

# model2= Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
# # model2.summary()

# x = model2.output
# x = Dense(len(labels), activation='softmax')(x)

# final_model = Model(inputs=model2.input, outputs=x)
# final_model.summary()

# new_model = Model(inputs=base_model.input, outputs=)

x_train, x_test, y_train, y_test = train_test_split(np.array(all_wave),np.array(y),stratify=y,
                                            test_size = 0.8,random_state=777,shuffle=True)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('tl_best_model_20_2ndSet.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

final_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
history = final_model.fit(x_train, y_train, epochs=10, callbacks=[es,mc], batch_size=32, validation_data=(x_test,y_test))

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8000, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 7988, 8)           112       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2662, 8)           0         
_________________________________________________________________
dropout (Dropout)            (None, 2662, 8)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2652, 16)          1424      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 884, 16)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 884, 16)          